## Integrantes
### Alejandro Orozco 2402036-7727
### Fernando Valencia 2401899-7727
### Carlos Botero 2400879-7727
### Brandon Rivas 2400430-7727

In [ ]:
# Instalación de librerías necesarias
!pip install -U transformers
!pip install torch

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
carpeta_historias = "/content/drive/MyDrive/Analitica en Salud/Negación"

In [ ]:
archivos = os.listdir(carpeta_historias)
print(len(archivos))

106


In [ ]:
historias = []
for archivo in archivos:
  ruta_completa = os.path.join(carpeta_historias, archivo)
  with open(ruta_completa, "r") as f:
    for linea in f:
      if linea.strip():
        historias.append(linea.strip())

In [ ]:
historias[0:10]

['PRIMERA CONSULTA DE ONCOLOGÍA MÉDICA.',
 'Antecedentes Personales:- Alergia a Fluconazol.',
 '- No HTA.',
 'No DM.',
 'No DL.',
 '- Niega habitos toxicos.- Candidiasis recurrentes- Iqx: ninguna.',
 'MEDICACIÓN- No medicación habitual.',
 'Muje de 59 años remitida desde oncología con Adenocarcinoma ductal infiltrante de mama izquierda, moderadamente diferenciado de 2 cm, intervenido mediante mastectomía radical izquierda el 20/06/1991.',
 '-Carcinoma lobulillar in situ residual de 2 cm en mama dcha, intervenida el 16/04/2003 mediante mastectomía radical derecha.',
 'Recidiva pulmonar y pleural confirmada con AP en el 2012.']

In [ ]:
from huggingface_hub import login
login()

Token has not been saved to git credential helper.


In [ ]:
from transformers import AutoTokenizer, AutoModelForTokenClassification, pipeline

# Cargar el tokenizer y el modelo
model_name = "anvorja/breast-cancer-biomedical-ner-sp-1"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForTokenClassification.from_pretrained(model_name)

# Crear pipeline de NER
ner_pipeline = pipeline("ner", model=model, tokenizer=tokenizer, aggregation_strategy="simple")



Device set to use cpu


In [ ]:
# Procesar las historias clínicas con el modelo NER
resultados_ner = []

for i, historia in enumerate(historias):
    entidades = ner_pipeline(historia)
    resultados_ner.append({
        "linea": historia,
        "entidades": entidades
    })

In [ ]:
for r in resultados_ner:
    print(f"\nTexto: {r['linea']}")
    for ent in r['entidades']:
        print(f"  → {ent['word']} ({ent['entity_group']}) [{ent['start']}-{ent['end']}]")



Texto: PRIMERA CONSULTA DE ONCOLOGÍA MÉDICA.
  → ONCOLOGÍA MÉDICA (CLINICAL_SERVICE) [20-36]

Texto: Antecedentes Personales:- Alergia a Fluconazol.
  → Alergia a Fluconazol (ALLERGIES) [26-46]

Texto: - No HTA.
  → No HTA (COMORBIDITY) [2-8]

Texto: No DM.
  → No DM (COMORBIDITY) [0-5]

Texto: No DL.
  → No DL (COMORBIDITY) [0-5]

Texto: - Niega habitos toxicos.- Candidiasis recurrentes- Iqx: ninguna.
  → Candidiasis recurrentes (GINECOLOGICAL_HISTORY) [26-49]

Texto: MEDICACIÓN- No medicación habitual.

Texto: Muje de 59 años remitida desde oncología con Adenocarcinoma ductal infiltrante de mama izquierda, moderadamente diferenciado de 2 cm, intervenido mediante mastectomía radical izquierda el 20/06/1991.
  → 59 años (AGE) [8-15]
  → remitida (OCURRENCE_EVENT) [16-24]
  → oncología (CLINICAL_SERVICE) [31-40]
  → Adenocarcinoma ductal infiltrante de mama izquierda (CANCER_CONCEPT) [45-96]
  → intervenido (OCURRENCE_EVENT) [134-145]
  → mastectomía radical izquierda (SURGERY) [155-18